In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
PATH = "../GSVData/Train"

In [2]:
batch_size = 512
image_size = (128, 96)
seed = 123

"""datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

train_data = datagen.flow_from_directory(
    "../GSVData/Train",
    target_size=image_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    "../GSVData/Test",
    target_size=image_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)"""

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "../GSVData/Train",
    image_size=image_size,
    color_mode='rgb',
    batch_size=batch_size,
    shuffle=True
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../GSVData/Test",
    color_mode='rgb',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)

class_names = train_data.class_names
print(class_names)

Found 397458 files belonging to 23 classes.
Found 132048 files belonging to 23 classes.
['Bangkok', 'Barcelona', 'Boston', 'Brussels', 'BuenosAires', 'Chicago', 'Lisbon', 'London', 'LosAngeles', 'Madrid', 'Medellin', 'Melbourne', 'MexicoCity', 'Miami', 'Minneapolis', 'OSL', 'Osaka', 'PRG', 'PRS', 'Phoenix', 'Rome', 'TRT', 'WashingtonDC']


In [3]:
train_data = train_data.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))

## 54% Model Similar to Paper

In [17]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50

def build_model(input_shape, num_classes):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Conv2D(1024, kernel_size=(2,2), activation="relu"),
        layers.Conv2D(512, kernel_size=(2,2), activation="relu"),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(.5),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation="softmax")
    ])

    optimizer = optimizers.Adam(learning_rate=.01)
    model.compile(optimizer='adam',
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])

    return model

model = build_model((320, 240, 3), 23)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 10, 8, 2048)       23587712  
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 7, 1024)        8389632   
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 6, 512)         2097664   
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                      

In [ ]:
# Assuming you have your model defined and compiled as 'model'
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Train the model
epochs = 20
model.fit(train_data, validation_data=validation_dataset, epochs=epochs)

## Model 2 Training Some Layers of ResNet

In [7]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50

def build_model(input_shape, num_classes):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    #training some layers
    for layer in base_model.layers[:150]:
        layer.trainable = False

    for layer in base_model.layers[150:]:
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.Conv2D(1024, kernel_size=(2,2), activation="relu"),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(.5),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax")
    ])

    optimizer = optimizers.Adam(learning_rate=.01)
    model.compile(optimizer='adam',
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])

    return model

model = build_model((128, 96, 3), 23)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 3, 2048)        23587712  
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 2, 1024)        8389632   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 23)                23575     
                                                      

In [8]:
# Assuming you have your model defined and compiled as 'model'
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Train the model
epochs = 7
model.fit(train_data, validation_data=validation_dataset, epochs=epochs)

Epoch 1/7
777/777 [==============================] - 620s 767ms/step - loss: 1.6466 - accuracy: 0.5128 - val_loss: 1.2373 - val_accuracy: 0.6165
Epoch 2/7
777/777 [==============================] - 580s 740ms/step - loss: 1.0586 - accuracy: 0.6693 - val_loss: 1.1024 - val_accuracy: 0.6595
Epoch 3/7
777/777 [==============================] - 579s 739ms/step - loss: 0.7677 - accuracy: 0.7542 - val_loss: 1.1290 - val_accuracy: 0.6707
Epoch 4/7
777/777 [==============================] - 606s 774ms/step - loss: 0.5337 - accuracy: 0.8252 - val_loss: 1.2749 - val_accuracy: 0.6659
Epoch 5/7
777/777 [==============================] - 577s 737ms/step - loss: 0.3634 - accuracy: 0.8795 - val_loss: 1.4029 - val_accuracy: 0.6689
Epoch 6/7
777/777 [==============================] - 610s 779ms/step - loss: 0.2551 - accuracy: 0.9147 - val_loss: 1.5979 - val_accuracy: 0.6658
Epoch 7/7
777/777 [==============================] - 615s 785ms/step - loss: 0.1922 - accuracy: 0.9353 - val_loss: 1.6371 - val_ac

In [4]:
mappings = {'Buenos Aires': 'BuenosAires', 'Los Angeles': 'LosAngeles', 'Mexico City': 'MexicoCity', 'Oslo': 'OSL', 'Prague': 'PRG',
                'Paris': 'PRS', 'Toronto':'TRT', 'Washington':'WashingtonDC'}
for city in os.listdir('street_images_plid'):
    if city in mappings:
        os.rename('street_images_plid/'+city, 'street_images_plid/'+mappings[city])

In [6]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "street_images_plid",
    color_mode='rgb',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

Found 3319 files belonging to 23 classes.


In [ ]:
model.save("resLast20.keras")

In [7]:
model = tf.keras.models.load_model('resLast20.keras')

In [ ]:
model.evaluate(test_dataset)

In [36]:
y_pred[0]

array([3.1054531e-10, 7.9473477e-13, 9.8248488e-01, 2.5718121e-04,
       3.4094158e-11, 7.6942742e-03, 2.2339540e-04, 6.8439944e-03,
       8.0232647e-11, 1.2131310e-03, 7.1725520e-12, 5.3281768e-04,
       4.1578627e-10, 1.3845216e-06, 7.3680672e-04, 9.7819512e-09,
       2.5877786e-10, 5.6131998e-13, 4.1028955e-15, 4.1832044e-07,
       1.1617857e-05, 1.7290515e-10, 1.1624794e-07], dtype=float32)

In [36]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50

def build_model(input_shape, num_classes):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    #training some layers
    for layer in base_model.layers[:140]:
        layer.trainable = False

    for layer in base_model.layers[140:]:
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.Conv2D(1024, kernel_size=(2,2), activation="relu"),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(.5),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax")
    ])

    optimizer = optimizers.Adam(learning_rate=.01)
    model.compile(optimizer='adam',
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])

    return model

model = build_model((128, 96, 3), 23)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 3, 2048)        23587712  
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 2, 1024)        8389632   
                                                                 
 global_average_pooling2d_5   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_5 (Dropout)         (None, 1024)              0         
                                                                 
 flatten_5 (Flatten)         (None, 1024)              0         
                                                                 
 dense_16 (Dense)            (None, 23)                23575     
                                                      

In [37]:
# Assuming you have your model defined and compiled as 'model'
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Train the model
epochs = 20
model.fit(train_data, validation_data=validation_dataset, epochs=epochs)

Epoch 1/20
777/777 [==============================] - 543s 692ms/step - loss: 1.7105 - accuracy: 0.4913 - val_loss: 1.2784 - val_accuracy: 0.6007
Epoch 2/20
777/777 [==============================] - 680s 870ms/step - loss: 1.1165 - accuracy: 0.6493 - val_loss: 1.1403 - val_accuracy: 0.6429
Epoch 3/20
777/777 [==============================] - 660s 843ms/step - loss: 0.8321 - accuracy: 0.7332 - val_loss: 1.1451 - val_accuracy: 0.6552
Epoch 4/20
777/777 [==============================] - 564s 721ms/step - loss: 0.5828 - accuracy: 0.8090 - val_loss: 1.2360 - val_accuracy: 0.6565
Epoch 5/20
777/777 [==============================] - 708s 906ms/step - loss: 0.3895 - accuracy: 0.8692 - val_loss: 1.4466 - val_accuracy: 0.6563
Epoch 6/20
777/777 [==============================] - 741s 948ms/step - loss: 0.2684 - accuracy: 0.9093 - val_loss: 1.5499 - val_accuracy: 0.6586
Epoch 7/20
777/777 [==============================] - 746s 954ms/step - loss: 0.1982 - accuracy: 0.9320 - val_loss: 1.6143 -

KeyboardInterrupt: 